In [ ]:
from domainbed.datasets.datasets import *
from domainbed.lib.fast_data_loader import *

## Test PACS_generated dataset ##

In [2]:
args = {"num_slices": 2, "fractal_weight": 0.2}

dataset = PACS_Generated(root="D:\Datasets\diffuseMix_augmented", args=args)

In [ ]:
x = dataset.__getitem__(1)